In [2]:
import numpy as np
import pandas as pd
import os
from typing import List, Union
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import matplotlib.cm as cm
plt.rcParams['font.size'] = 20
%matplotlib qt
from scipy import signal, optimize
from sys import exit

In [3]:
# enumerating temps
temperatures = [100, 105, 10, 110, 115, 120, 125, 130, 135, 13, 140, 145, 150, 155, 15, 160, 165, 170, 175, 180, 185, 190, 195, 200, 205, 20, 210, 215, 220, 225, 230, 235, 240, 250, 25, 260, 270, 27, 280, 290, 300, 30, 35, 40, 45, 50, 53, 55, 57, 5, 60, 63, 65, 70, 75, 80, 85, 8, 90, 95, 245]
temperatures.sort()
n_temps = len(temperatures)

def Temp(t: int) -> int:
    """
    This function returns the index of the temp
    """
    return temperatures.index(t)

    
norm = Normalize(vmin=min(temperatures), vmax=max(temperatures), clip=True)

mapper = cm.ScalarMappable(norm=norm, cmap='coolwarm')

temp_color = [(mapper.to_rgba(i)) for i in temperatures]

# Data Axes
## Order

| 0 | 1 | 2 | 3 | 4 |
| --- | --- | --- | --- | --- |
| power | polarization | temp | column (reflectivity, kerr, delay) | points |


## Mapping

### power:
| 0 | 1 | 2 |
| --- | --- | --- |
| 10mW | 25mW | 48mW |

### polarization:
| 0 | 1 | 2 |
| --- | --- | --- |
| right | left | linear |

### temp
| 0-60 |
| --- |
| relevant uniqe temps as enumerated in the cell above |

### Column
| 0 | 1 | 2 |
| --- | --- | --- |
| delay | reflectivity | kerr |

In [4]:
Dims = (3,3,61,3,3200)

def load_data(folders: List[str]) -> np.ndarray:
    
    # Dimentions/axes: power, polarization, temp, column (reflectivity, kerr, delay), points
    data = np.empty(Dims)
    
    
    #temps = []  # for finding uniqe temps
    
    
    #file_counter = 0  # for counting total number of files
    
    
    for folder in folders:
        for root, subdirs, files in os.walk(folder):            
            for fname in files:
                if fname[:9] == "PumpProbe":
                    #file_counter += 1  # for counting total number of files
                    splitted_str = fname.split("_")
                    ########### power ############
                    power = int(splitted_str[5][:2])
                    if power == 40:
                        power = 25
                    
                    if power == 10:
                        axis_power = 0
                    elif power == 25:
                        axis_power = 1
                    elif power == 48:
                        axis_power = 2
                    ##############################
                    
                    
                    ############ temp ############
                    temp = int(splitted_str[4][:-1])
                    #if temp not in temps:  # for finding uniqe temps
                        #temps.append(temp)
                    
                    axis_temp = temperatures.index(temp)
                    ##############################
                    
                    
                    ############ polarization ############
                    polarization = splitted_str[-1]
                    
                    if polarization == "right":
                        axis_pol = 0
                    elif polarization == "left":
                        axis_pol = 1
                    elif polarization == "lin":
                        axis_pol = 2
                    ######################################
                    
                    
                    measurement = pd.read_csv(os.path.join(root, fname),header = 9).to_numpy()
                    measurement = np.flip(measurement, axis=0)
                    
                    data[axis_power,axis_pol,axis_temp,0,:] = measurement[:, 0]
                    data[axis_power,axis_pol,axis_temp,1,:] = measurement[:, 14]
                    data[axis_power,axis_pol,axis_temp,2,:] = measurement[:, 6]                    

    return data


In [5]:
def plot_single(full_data, y_axis, power, pol, temp, x_axis=0, slicer=(0,-4)):
    """
    This function takes the relevent axes and plots a single line
    The x axis is usually Delay
    """
    plt.figure(figsize=(16,9))

    slc = slice(slicer[0], slicer[1])
    
    if y_axis == "ref":
        y_axis = 1
        plt.ylabel("Ref.")
    elif y_axis == "kerr":
        y_axis = 2
        plt.ylabel("Kerr")
        
    if pol == "right":
        pol = 0
        pol_label = "Right"
    elif pol == "left":
        pol = 1
        pol_label = "Left"
    elif pol == "linear":
        pol = 2
        pol_label = "Linear"
        
    if pol == 0:
        pol_label = "Right"
    elif pol == 1:
        pol_label = "Left"
    elif pol == 2:
        pol_label = "Linear"

    if power == 10 or power == "10mw" or power == "10mW":
        power = 0
        power_label = "10mW"
    elif power == 25 or power == "25mw" or power == "25mW":
        power = 1
        power_label = "25mW"
    elif power == 48 or power == "48mw" or power == "48mW":
        power = 2
        power_label = "48mW"
        
    if power == 0:
        power_label = "10mW"
    elif power == 1:
        power_label = "25mW"
    elif power == 2:
        power_label = "48mW"
        
        
    temp_label = f"{temperatures[temp]}K"
    
    print(slc)
    x = full_data[power,pol,temp,x_axis,:][slc]
    y = full_data[power,pol,temp,y_axis,:][slc]
    
    plt.xlabel("Delay [ps]")
    plt.title(pol_label + ", " + temp_label + ", " + power_label)
    plt.plot(x, y)

In [6]:
def plot_axis(full_data, y_axis, axis_to_plot, other_2_axes:List[any], x_axis=0, slicer=(0,-4), axis_slicer=(0, -1)):
    """
    - This function takes the relevent axes and plots every thing in a certain axis
    - The x axis is usually Delay
    - The other two axes are in order. for example:
        if you want to plot all the temperatures: axis_to_plot=="Temp"
        Temperature is the last of the relevant axis (out of power, pol and temp)
        Hence other_2_axes=(0,1) means (power axis=0=10mW, pol axis=1=left)
        
        if you wanted to plot all the powers
        other_2_axes=(0,1) means (pol axis=0=right, temp axis=1=Temp(1))
    """
    plt.figure(figsize=(16,9))

    if len(other_2_axes) != 2:
        print("There are only 2 remaining axes")
        exit(1)
        
    slc = slice(slicer[0], slicer[1])
    ax_slc = slice(axis_slicer[0], axis_slicer[1])
    
    if y_axis == "ref":
        y_axis = 1
        plt.ylabel("Ref.")
    elif y_axis == "kerr":
        y_axis = 2
        plt.ylabel("Kerr")

        
    if axis_to_plot=="Temp" or axis_to_plot=="temp" or axis_to_plot=="t" or axis_to_plot=="T":  # if tou want to plot all temps
        
        power, pol = other_2_axes
        
        if pol == "right":
            pol = 0
            pol_label = "Right"
        elif pol == "left":
            pol = 1
            pol_label = "Left"
        elif pol == "linear":
            pol = 2
            pol_label = "Linear"

        if pol == 0:
            pol_label = "Right"
        elif pol == 1:
            pol_label = "Left"
        elif pol == 2:
            pol_label = "Linear"

        if power == 10 or power == "10mw" or power == "10mW":
            power = 0
            power_label = "10mW"
        elif power == 25 or power == "25mw" or power == "25mW":
            power = 1
            power_label = "25mW"
        elif power == 48 or power == "48mw" or power == "48mW":
            power = 2
            power_label = "48mW"

        if power == 0:
            power_label = "10mW"
        elif power == 1:
            power_label = "25mW"
        elif power == 2:
            power_label = "48mW"
            
            
        for t in temperatures[ax_slc]:
            
            x = full_data[power,pol,Temp(t),x_axis,:][slc]
            y = full_data[power,pol,Temp(t),y_axis,:][slc]
            
            plt.plot(x, y, label=f"{t}K")
            
            
        plt.xlabel("Delay [ps]")
        plt.title(pol_label + ", " + power_label)
        plt.legend(loc="best")
        
        
    

In [7]:
def plot_fft_single(full_data, y_axis, power, pol, temp, slicer=(0,-4)):
    """
    This function takes the relevent axes and plots a single line
    The x axis is usually Delay
    """
    plt.figure(figsize=(16,9))

    slc = slice(slicer[0], slicer[1])
    
    if y_axis == "ref":
        y_axis = 1
        ylabel = "Ref."
    elif y_axis == "kerr":
        y_axis = 2
        ylabel = "Kerr"
        
    if pol == "right":
        pol = 0
        pol_label = "Right"
    elif pol == "left":
        pol = 1
        pol_label = "Left"
    elif pol == "linear":
        pol = 2
        pol_label = "Linear"
        
    if pol == 0:
        pol_label = "Right"
    elif pol == 1:
        pol_label = "Left"
    elif pol == 2:
        pol_label = "Linear"

    if power == 10 or power == "10mw" or power == "10mW":
        power = 0
        power_label = "10mW"
    elif power == 25 or power == "25mw" or power == "25mW":
        power = 1
        power_label = "25mW"
    elif power == 48 or power == "48mw" or power == "48mW":
        power = 2
        power_label = "48mW"
        
    if power == 0:
        power_label = "10mW"
    elif power == 1:
        power_label = "25mW"
    elif power == 2:
        power_label = "48mW"
        
        
    temp_label = f"{temperatures[temp]}K"
    
    x_delay = full_data[0,0,0,0,:][slc]
    
    T = np.diff(x_delay)[0]
    
    x = np.fft.rfftfreq(x_delay.shape[0], T)

    y = np.fft.rfft(full_data[power,pol,temp,y_axis,:][slc])
        
    plt.ylabel("Amplitude A.U.")
    plt.xlabel(r"$\omega$ [THz]")
    plt.title(ylabel + ", " + pol_label + ", " + temp_label + ", " + power_label)
    plt.plot(x, np.abs(y), ls="-", marker='.', ms=10)

In [8]:
def get_fft(full_data) -> (np.ndarray, np.ndarray):
    
    x_delay = full_data[0,0,0,0,:]
    
    T = np.diff(x_delay)[0]
    
    x = np.fft.rfftfreq(x_delay.shape[0], T)

    y = np.fft.rfft(full_data)
    
    return x, y

In [9]:
data = load_data([r"Data/polarization temp run10mW", r"Data/polarization temp run25mW", r"Data/polarization temp run48mW"])
print(data.shape)

(3, 3, 61, 3, 3200)


In [10]:
plot_single(data, 'ref', 0, 1, Temp(50), slicer=(430,-800))
plot_fft_single(data, 'ref', 0, 1, Temp(50), slicer=(430,-800))
plot_fft_single(data, 'ref', 0, 1, Temp(50), slicer=(430,-300))

slice(430, -800, None)


In [11]:
x_fft, y_fft = get_fft(data[...,430:-4])
print(x_fft.shape)
print(y_fft.shape)
print(Dims)

(1384,)
(3, 3, 61, 3, 1384)
(3, 3, 61, 3, 3200)


# Get Phonons - Kerr

In [12]:
phonon_Dims = (3,3,61,3,10)
# for every power, pol and temp there are phonons (supposedly) for the kerr and ref and maximum 4 phonons, each phonon is a x and y point in the fourier domain
phonons_array_kerr = np.zeros(phonon_Dims) # (power, pol, temp, phonon x/y/noise, num phonon)

for power in range(3):
    for pol in [0, 1, 2]:
        for temp in range(61):
            for y_axis in [2]:
                
                if y_axis == 1:
                    ylabel = "Ref."
                elif y_axis == 2:
                    ylabel = "Kerr"

                if pol == "right":
                    pol = 0
                    pol_label = "Right"
                elif pol == "left":
                    pol = 1
                    pol_label = "Left"
                elif pol == "linear":
                    pol = 2
                    pol_label = "Linear"

                if pol == 0:
                    pol_label = "Right"
                elif pol == 1:
                    pol_label = "Left"
                elif pol == 2:
                    pol_label = "Linear"

                if power == 10 or power == "10mw" or power == "10mW":
                    power = 0
                    power_label = "10mW"
                elif power == 25 or power == "25mw" or power == "25mW":
                    power = 1
                    power_label = "25mW"
                elif power == 48 or power == "48mw" or power == "48mW":
                    power = 2
                    power_label = "48mW"

                if power == 0:
                    power_label = "10mW"
                elif power == 1:
                    power_label = "25mW"
                elif power == 2:
                    power_label = "48mW"

                temp_label = f"{temperatures[temp]}K"
                
                label = ylabel + ", " + pol_label + ", " + temp_label + ", " + power_label

                if temperatures[temp] < 80:
                    peaks, _ = signal.find_peaks(np.abs(y_fft[power,pol,temp,y_axis,:]), prominence=0.0015)
                else:
                    peaks, _ = signal.find_peaks(np.abs(y_fft[power,pol,temp,y_axis,:]), prominence=0.006)
                    
                plt.plot(x_fft, np.abs(y_fft[power,pol,temp,y_axis,:]), label=label, c=temp_color[temp], marker='o', ms=3, lw=1)
                
                
                #plt.hlines(    np.average( np.abs(y_fft[power,pol,temp,y_axis,:])[:600],)   , x_fft.min(), x_fft.max(), colors='g', ls='-')
                #plt.legend(loc='best')
                
                #print(x_fft[peaks].shape)
                #print(np.abs(y_fft[power,pol,temp,y_axis,:][peaks]).shape)
                
                condition = (x_fft[peaks] > 2) & (x_fft[peaks] < 5)
                
                x_phonons = np.zeros(phonons_array_kerr.shape[-1])
                y_phonons = np.zeros(phonons_array_kerr.shape[-1])
                ymid = np.ones(phonons_array_kerr.shape[-1])
                xmid = np.ones(phonons_array_kerr.shape[-1])

                n_phonons = x_fft[peaks][condition].shape[0]
                
                #print(n_phonons)
                #print(peaks, peaks+2)
                
                x_phonons[:n_phonons] = x_fft[peaks][condition]
                y_phonons[:n_phonons] = np.abs(y_fft[power,pol,temp,y_axis,:][peaks])[condition]
                
                #print(x_phonons , y_phonons)
                
                
                # get backround around peaks
                points_away = 6
                
                bgx1 = x_fft[peaks-points_away][condition]
                bgy1 = np.abs(y_fft[power,pol,temp,y_axis,:][peaks-points_away])[condition]
                plt.plot(bgx1, bgy1, "x" , c='g')
                
                bgx2 = x_fft[peaks+points_away][condition]
                bgy2 = np.abs(y_fft[power,pol,temp,y_axis,:][peaks+points_away])[condition]
                plt.plot(bgx2, bgy2, "x" , c='g')
                
                xmid[:n_phonons] = (bgx2 + bgx1)/2
                ymid[:n_phonons] = (bgy2 + bgy1)/2
                plt.plot(xmid, ymid, "x" , c='k', ms=10)

                #populate array

                phonons_array_kerr[power,pol,temp,0,:] = x_phonons
                #print(peaks.any(), y_phonons, ymid)
                phonons_array_kerr[power,pol,temp,1,:] = y_phonons
                phonons_array_kerr[power,pol,temp,2,:] = ymid
                plt.plot(x_phonons, y_phonons, "x" , c='r')
                
                plt.xlabel(r"$\omega$ [THz]")
                plt.ylabel("Amplitude A.U.")
                

# Plot Phonons - Kerr

In [13]:
for power in range(3):
    for pol in range(3):
        for y_axis in [1]:
            for phonon in range(phonon_Dims[-1]):

                if y_axis == 0:
                    ylabel = "Ref."
                elif y_axis == 1:
                    ylabel = "Kerr"

                if pol == "right":
                    pol = 0
                    pol_label = "Right"
                elif pol == "left":
                    pol = 1
                    pol_label = "Left"
                elif pol == "linear":
                    pol = 2
                    pol_label = "Linear"

                if pol == 0:
                    pol_label = "Right"
                elif pol == 1:
                    pol_label = "Left"
                elif pol == 2:
                    pol_label = "Linear"

                if power == 10 or power == "10mw" or power == "10mW":
                    power = 0
                    power_label = "10mW"
                elif power == 25 or power == "25mw" or power == "25mW":
                    power = 1
                    power_label = "25mW"
                elif power == 48 or power == "48mw" or power == "48mW":
                    power = 2
                    power_label = "48mW"

                if power == 0:
                    power_label = "10mW"
                elif power == 1:
                    power_label = "25mW"
                elif power == 2:
                    power_label = "48mW"

                temp_label = f"{temperatures[temp]}K"
                
                freq = f"{phonons_array_kerr[power,pol,:,0,phonon].max():.3f}"

                label = ylabel + ", " + pol_label + ", " + power_label + ", " + freq + "THz"
                
                if not phonon:  # if phonon=0 => lower freq phonon
                    color = 'b'
                elif phonon: # if phonon=1 => high freq phonon
                    color = 'r'

                plt.plot(temperatures, phonons_array_kerr[power,pol,:,1,phonon], label=label, ls='None', marker='o', ms=7, c=color)

                #plt.legend(loc='best')
                
                plt.xlabel("T [K]")
                plt.ylabel("Amplitude A.U.")
                #plt.xlim((0, 300))
                #plt.ylim((0, 0.02))

# Get Phonons - Ref.

In [14]:
phonon_Dims = (3,3,61,3,20)
phonons_array_ref = np.zeros(phonon_Dims) # (power, pol, temp, phonon x/y/noise, num phonon)

n_phonons_lst = []

for power in [1, 0, 2]:
    for pol in [0, 1, 2]:
        for temp in range(61):
            for y_axis in [1]:
                
                if y_axis == 1:
                    ylabel = "Ref."
                elif y_axis == 2:
                    ylabel = "Kerr"

                if pol == "right":
                    pol = 0
                    pol_label = "Right"
                elif pol == "left":
                    pol = 1
                    pol_label = "Left"
                elif pol == "linear":
                    pol = 2
                    pol_label = "Linear"

                if pol == 0:
                    pol_label = "Right"
                elif pol == 1:
                    pol_label = "Left"
                elif pol == 2:
                    pol_label = "Linear"

                if power == 10 or power == "10mw" or power == "10mW":
                    power = 0
                    power_label = "10mW"
                elif power == 25 or power == "25mw" or power == "25mW":
                    power = 1
                    power_label = "25mW"
                elif power == 48 or power == "48mw" or power == "48mW":
                    power = 2
                    power_label = "48mW"

                if power == 0:
                    power_label = "10mW"
                elif power == 1:
                    power_label = "25mW"
                elif power == 2:
                    power_label = "48mW"

                temp_label = f"{temperatures[temp]}K"
                
                label = ylabel + ", " + pol_label + ", " + temp_label + ", " + power_label

                diff = np.diff(x_fft)[0]
                
                if temp < 15:
                    peaks, _ = signal.find_peaks(np.abs(y_fft[power,pol,temp,y_axis,:]), prominence=0.006, distance=10)
                elif temp > 15 and temp < 30:
                    peaks, _ = signal.find_peaks(np.abs(y_fft[power,pol,temp,y_axis,:]), prominence=0.0085, distance=10)
                elif temp > 30 and temp < 45:
                    peaks, _ = signal.find_peaks(np.abs(y_fft[power,pol,temp,y_axis,:]), prominence=0.01, distance=12)
                else:
                    peaks, _ = signal.find_peaks(np.abs(y_fft[power,pol,temp,y_axis,:]), prominence=0.01, distance=12)

                    
                plt.plot(x_fft, np.abs(y_fft[power,pol,temp,y_axis,:]), label=label, c=temp_color[temp], marker='o', ms=3, lw=1)
                
                
                #plt.hlines(    np.average( np.abs(y_fft[power,pol,temp,y_axis,:])[:600],)   , x_fft.min(), x_fft.max(), colors='g', ls='-')
                #|plt.legend(loc='best')
                
                #print(x_fft[peaks].shape)
                #print(np.abs(y_fft[power,pol,temp,y_axis,:][peaks]).shape)
                
                condition = (x_fft[peaks] > 3) & (x_fft[peaks] < 4.5)
                
                x_phonons = np.zeros(phonons_array_ref.shape[-1])
                y_phonons = np.zeros(phonons_array_ref.shape[-1])
                ymid = np.ones(phonons_array_ref.shape[-1])
                xmid = np.ones(phonons_array_ref.shape[-1])

                n_phonons = x_fft[peaks][condition].shape[0]
                n_phonons_lst.append(n_phonons)
                
                #print(n_phonons)
                #print(peaks, peaks+2)
                
                x_phonons[:n_phonons] = x_fft[peaks][condition]
                y_phonons[:n_phonons] = np.abs(y_fft[power,pol,temp,y_axis,:][peaks])[condition]
                
                #print(x_phonons , y_phonons)
                
                
                # get backround around peaks
#                 points_away = 6
                
#                 bgx1 = x_fft[peaks-points_away][condition]
#                 bgy1 = np.abs(y_fft[power,pol,temp,y_axis,:][peaks-points_away])[condition]
#                 plt.plot(bgx1, bgy1, "x" , c='g')
                
#                 bgx2 = x_fft[peaks+points_away][condition]
#                 bgy2 = np.abs(y_fft[power,pol,temp,y_axis,:][peaks+points_away])[condition]
#                 plt.plot(bgx2, bgy2, "x" , c='g')
                
#                 xmid[:n_phonons] = (bgx2 + bgx1)/2
#                 ymid[:n_phonons] = (bgy2 + bgy1)/2
#                 plt.plot(xmid, ymid, "x" , c='k', ms=10)

                #populate array

                phonons_array_ref[power,pol,temp,0,:] = x_phonons
                #print(peaks.any(), y_phonons, ymid)
                phonons_array_ref[power,pol,temp,1,:] = y_phonons
                phonons_array_ref[power,pol,temp,2,:] = ymid
                plt.plot(x_phonons, y_phonons, "x" , c='r')
                
                plt.xlabel(r"$\omega$ [THz]")
                plt.ylabel("Amplitude A.U.")
                
print(np.asarray(n_phonons_lst).mean())

1.7777777777777777


# Plot Phonons - Ref.

In [15]:
for power in range(3):
    for pol in range(3):
        for y_axis in [1]:
            for phonon in range(phonon_Dims[-1]):

                if y_axis == 0:
                    ylabel = "Ref."
                elif y_axis == 1:
                    ylabel = "Kerr"

                if pol == "right":
                    pol = 0
                    pol_label = "Right"
                elif pol == "left":
                    pol = 1
                    pol_label = "Left"
                elif pol == "linear":
                    pol = 2
                    pol_label = "Linear"

                if pol == 0:
                    pol_label = "Right"
                elif pol == 1:
                    pol_label = "Left"
                elif pol == 2:
                    pol_label = "Linear"

                if power == 10 or power == "10mw" or power == "10mW":
                    power = 0
                    power_label = "10mW"
                elif power == 25 or power == "25mw" or power == "25mW":
                    power = 1
                    power_label = "25mW"
                elif power == 48 or power == "48mw" or power == "48mW":
                    power = 2
                    power_label = "48mW"

                if power == 0:
                    power_label = "10mW"
                elif power == 1:
                    power_label = "25mW"
                elif power == 2:
                    power_label = "48mW"

                temp_label = f"{temperatures[temp]}K"
                
                freq = f"{phonons_array_ref[power,pol,:,0,phonon].max():.3f}"

                label = ylabel + ", " + pol_label + ", " + power_label + ", " + freq + "THz"
                
                if not phonon:  # if phonon=0 => lower freq phonon
                    color = 'b'
                elif phonon: # if phonon=1 => high freq phonon
                    color = 'r'

                plt.plot(temperatures[:40], phonons_array_ref[power,pol,:40,1,phonon], label=label, ls='None', marker='o', ms=7, c=color)

                #plt.legend(loc='best')
                
                plt.xlabel("T [K]")
                plt.ylabel("Amplitude A.U.")

In [78]:
def fit_func(x, tau1, tau2, a, b, scale):
    return scale*(a*np.exp(x/tau1) + b*np.exp(x/tau2))

def fit_func1(x, w1, a, ph):
    return -2.61168538e-04*(5.49949903e+00*np.exp(x/-3.61657463e+00) + 1.50727467e+00*np.exp(x/4.18558584e+01) + a*(np.sin(w1*x - ph)))

def fit_func2(x, a, tau1, w1, w2, ph1, ph2):
    return -2.61168538e-04*(5.49949903e+00*np.exp(x/-3.61657463e+00) + 1.50727467e+00*np.exp(x/4.18558584e+01)) + a*np.exp(x/tau1)*(np.sin(w1*x - ph1) + np.sin(w2*x - ph2))


In [92]:
#plot_single(data, 'ref', 0, 1, Temp(50), slicer=(0,-4))
x = data[1,0,Temp(20),0,300:-1000]
y = data[1,0,Temp(20),1,300:-1000]

p_start = (-3.61657463e+00, 4.18558584e+01, 5.49949903e+00, 1.50727467e+00, -2.61168538e-04)
y_sub = y - fit_func(x, *p_start)
popt, pcov = optimize.curve_fit(fit_func2, x, y, p0=(0.01, -1, 3.14*(2*np.pi), 4*(2*np.pi) , 0.0, 0), bounds=([-5, -10, 1*(2*np.pi),1*(2*np.pi), 0, 0],[5, 0, 5*(2*np.pi), 5*(2*np.pi), np.pi/2, np.pi/2]))#bounds=(0,[5, np.inf])
#popt, pcov = optimize.curve_fit(fit_func2, x, y,)
#yfit = fit_func(x, -0.11657463e-01, -4.18558584e+00, 1.49949903e+00, 3.50727467e+00, -2.61168538e-04)

yfit = fit_func2(x, *popt)
print(popt/(2*np.pi))

plt.plot(x,y)
plt.plot(x,yfit) #+(0.00004*np.sin(20*x - 8.5))

[ 1.34833929e-05 -1.18049209e+00  3.40206312e+00  4.13712399e+00
  6.68325130e-02  1.55166075e-01]


In [ ]:
#plot_single(data, y_axis="ref", power=0, pol=0, temp=Temp(300))

In [ ]:
#plot_axis(data, y_axis="ref", axis_to_plot="T", other_2_axes=(0, 'right'), axis_slicer=(0, -1))

In [ ]:

#N = int(np.round(np.abs(np.max(x_delay)-np.min(x_delay))/T)+1)



#plot_single(data, y_axis="ref", power=25, pol=0, temp=Temp(10))
